In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
def match_est_MF(pitch_min=85.5, pitch_max=88.5, n_template = np.logspace(2,5,11), n_signal='None'):
    
    path2scores = Path.home()/'group'/'project'/'results'/'matched_filter'/\
    'scores'/'220922_dl_test_data_mf_scores.npz'
    
    scores_file = np.load(path2scores)
    
    results = Path.home()/'group'/'project'/'results'/'deepfiltering_paper'/'match_est'/'pmin855_pmax885_'
    
    ideal_scores_signal = scores_file['signal_ideal_scores']
    #scores = scores_file['scores']
    sig_metadata, temp_metadata = {}, {}
    
    print('Signal Metadata')
    sig_metadata['signal_energy'] = scores_file['signal_energy'][:]
    sig_metadata['signal_pitch'] = scores_file['signal_pitch'][:]
    
    print('Template Metadata')
    temp_metadata['template_energy'] = scores_file['template_energy'][:]
    temp_metadata['template_pitch'] = scores_file['template_pitch'][:]
    
    print('Dataframe')
    sig_metadata = pd.DataFrame(sig_metadata)
    temp_metadata = pd.DataFrame(temp_metadata)
    
    print('Parse Dataframe')
    signal_inds = np.array(sig_metadata[
        (sig_metadata['signal_pitch']>=pitch_min)&(sig_metadata['signal_pitch']<=pitch_max)
    ].index.array)
    
    # reduce the number of signals to speed up analysis
    if n_signal != 'None':
        signal_inds = np.random.choice(signal_inds, size=n_signal)
    
    temp_inds = np.array(temp_metadata[
        (temp_metadata['template_pitch']>=pitch_min)&(temp_metadata['template_pitch']<=pitch_max)
    ].index.array)
    
    ideal_scores_signal = ideal_scores_signal[signal_inds]
    
    no_nan_signal = ~np.isnan(ideal_scores_signal)
    ideal_scores = ideal_scores_signal[no_nan_signal]
    shape = scores_file['scores'].shape
    
    mean_match = np.zeros(n_template.size) 
    match_histograms = []
    #n_template = np.array([2,5,10])
    print('Starting Match Calculation')
    for i, number in enumerate(n_template):
        indicies = np.int32(np.linspace(0, shape[1]-1, np.int32(number)))
        signal_grid, template_grid = np.meshgrid(signal_inds[no_nan_signal], indicies)
        
        #print('Signals')
        #print(signal_grid)
        #print('Templates')
        #print(template_grid)
        
        select_scores = scores_file['scores'][signal_grid, template_grid] # max over the templates
        
        #print(select_scores.shape)
        
        temp_where = np.where(np.isnan(select_scores))
        select_scores[temp_where[0], temp_where[1]] = 0
        
        select_scores = select_scores.max(axis=0)
        
        #select_scores_mean = select_scores.mean(axis=0)
        match = np.mean(select_scores / ideal_scores)
        mean_match[i] = match
        match_histograms.append(np.histogram(select_scores/ideal_scores, bins=201))
        
        #if i % 5 == 4:
        print(f'{i+1} / {n_template.size}. Match = {match}, Templates = {np.int32(number)}')
        
        #input('Enter')
            
    
    return mean_match, n_template, match_histograms
        
        
    

In [ ]:
mean_match, n_template, match_histograms = match_est_MF(
    pitch_min=85.5,
    pitch_max=88.5,
    n_signal='None', 
    n_template=np.logspace(2,5.2,41)
)

Signal Metadata
Template Metadata
Dataframe
Parse Dataframe
Starting Match Calculation
1 / 31. Match = 0.10301767564680869, Templates = 100
2 / 31. Match = 0.11798335428757191, Templates = 125
3 / 31. Match = 0.13859537231267766, Templates = 158
4 / 31. Match = 0.15781382694478288, Templates = 199
5 / 31. Match = 0.18638271698498196, Templates = 251
6 / 31. Match = 0.2119883535044223, Templates = 316
7 / 31. Match = 0.24108512347544014, Templates = 398
